# Relatorio Técnico Referente a Matéria de Técnicas de Pesquisa Experimental
  
**Tema:** Analise da Variação de Parâmetros na Impressão 3D de Filamentos Poliméricos  
**Profº:** Pedro Bastos Costa  
**Universidade Federal de Minas Gerais - UFMG**

**Membros:**
* Guilherme de Paula Rúbio  
* Matheus Ungaretti Borges  
* Daniel Affonso Vasconcelos  

## *Importação de Bibliotecas*

In [1]:
import numpy as np 
import pandas as pd 
import scipy.stats as st 
import matplotlib.pyplot as plt 

from math import sqrt

%matplotlib inline

## ***Fuções Auxiliares Criadas***  

* **Função**: Teste de hipótese bicaudal, distribuição F

In [2]:
def F_bicaudal_hypoteses (F :float, alpha :float, dn :float, dd :float):
    '''
        Função de teste de hipótese de um teste bicaudal do tipo F
            Entrada:
               * F: Valor calculado de F_0;
               * alpha: nivel de confiança total;
               * dn: graus de liberdade do numerador;
               * dd: graus de liberdade do denominado.

            Saídas:
               * Impressão das Respostas.

            Retorno: Nenhum
    '''
    min = st.f.ppf(alpha/2, dn, dd)         # Limite Superior
    max = st.f.ppf(1-(alpha/2), dn, dd)     # Limite Inferior

    print('O valor da estatísca cálcula F0 é: {:.4f}'.format(F))
    print('O limites da distribuição para {} de {:.2f} é: [{:.4f}:{:.4f}]'.format('\u0251', alpha, min, max))
    print('Portanto,',end=' ')

    if (F<min) or (F>max):
        print('\033[33mREJEITA-SE\033[m a hipótese nula!')
        print('p-valor: {:.4f}'.format( st.f.cdf(F, dn, dd) if (F<min) else (1-st.f.cdf(F, dn, dd)) ))
    else:
        print('\033[31mFALHA EM REJEITAR\033[m a hipótese nula!')

## *Importação dos dados*
Todos os dados utilizados nessa análise estão disponibilizados na plataforma Kaggle. Caso queria acessar a fonte dos dados utilizados basta [clicar aqui.](https://www.kaggle.com/afumetto/3dprinter?select=data.csv)  

### Contexto  
Os dados utilizados são oriundos de uma pesquisa desenvolvida pelo Departamento de Engenharia Mecânica da TR/Selcuk University.  
Essa pesquisa visava avaliar quais os parâmetros de impressão interferem na qualidade de impressão de peças, precisão e rigidez.  
Nos dados apresentados existem nove parâmetro de configuração (entradas), e as medições de três parâmetros de saída.  

### Materiais e Métodos  
* **Impressora:** Ultimaker S5 3-D 
* **Teste de materiais e resistência:** Sincotec GMBH, capacidade de tração - 20kN.

### Conteúdo dos dados
#### Entradas - Parâmetros de configuração:  
* Altura de camada (*Layer Height*) \[mm]  
* Espessura de casca (*Wall Thickness*) \[mm]  
* Densidade de preenchimento (*Infill Density*) \[%]  
* Padrão de preenchimento (*Infill Pattern*)  
* Temperatura do bico de extrusão (*Nozzle Temperature*) \[°C]  
* Temperetura da mesa de impressão (*Bed Temperature*) \[°C]  
* Velocidade de impressão (*Print Speed*) \[mm/s]  
* Material (*Material*)  
* Velocidade do vetilador da extrusora (*Fan Speed*) \[%]  

#### Saídas  
* Rugosidade (*Roughness*) \[µm]
* Tensão de ruptura (*Ultimate Tension Strenght*) \[MPa]
* Alongação (*Elongation*) \[%]  

Abaixo importamos os dados para a análise.  

In [3]:
dados = pd.read_csv('datasets\data_3D_print.csv')   # Importa o DataFrame
dados.head()     # Apresenta as 5 primeiras linhas do DataFrame

,layer_height,wall_thickness,infill_density,infill_pattern,nozzle_temperature,bed_temperature,print_speed,material,fan_speed,roughness,tension_strenght,elongation
0,0.02,8,90,grid,220,60,40,abs,0,25,18,1.2
1,0.02,7,90,honeycomb,225,65,40,abs,25,32,16,1.4
2,0.02,1,80,grid,230,70,40,abs,50,40,8,0.8
3,0.02,4,70,honeycomb,240,75,40,abs,75,68,10,0.5
4,0.02,6,90,grid,250,80,40,abs,100,92,5,0.7


## *Análise das Saídas*  
Como temos três saidas iremos analisa-las separadamente.  

### Saida Rugosidade:  

#### ***Influência do material***  
Umas das primeiras hipóteses que desejamos saber do ponto de vista da rugosidade é se o material afeta estatisticamente seu valor, para isso separaremos as saídas de rugosidade por material para realizar uma análise de experimento com um único fator.

In [4]:
dados_roughness_material = dados[['material','roughness']]
dados_roughness_material.head()

,material,roughness
0,abs,25
1,abs,32
2,abs,40
3,abs,68
4,abs,92


Com os dados separados podemos observar que nosso problema é dividido em dois materiais, ou seja, dois níveis (PLA e ABS). Primeiramente vamos analisar quantas observações de cada materiais nos temos.  

In [5]:
dados_roughness_material['material'].value_counts()

pla    25
abs    25
Name: material, dtype: int64

Como podemos observa o número de observações de cada material é igual.
***
#### Teoria 📖:  
Supomos que nosso valor de saída rugosidade varia de acordo com o material utilizado, o que significaria que ela teria o seguinte comportamento:  
$$
\mathsf{
    Y_{ij}= \mu + \tau_{i} + \epsilon_{ij}
}
\left\{\displaystyle
    \begin{array}{l}
        \mathsf{i=0, 1}\\
        \mathsf{j=0, 1, 2, 3, \dots, 25}
    \end{array}
\right.
$$  
Onde:
   * $\mu$: média global;  
   * $\tau_{i}$: fator de alteração referente ao material utilizado;
   * $\epsilon_{ij}$: erro aleatorio da medição  
***  
Para garantirmos que o realmente o material interfere na rugosidade medida nas nossas peças temos que garantir que o valor de $\tau$ de cada material seja diferente de zero. Logo temos que fazer um teste assumindo as seguintes hipóteses:  
$$
\left\{
    \begin{array}{}
        \mathsf{\mathit{H_0}: \tau_{0}=\tau_{1}=0}\\
        \mathsf{\mathit{H_1}: \tau_{0}\not=\tau_{1}\not=0}
    \end{array}
\right.
$$  
Para isso faremos um teste ANOVA. Para isso assumimos que o modelo assumi uma estatística F, assim:  
$$
\mathsf{
    F_0=\dfrac{\dfrac{SQ_{tratamentos}}{a-1}}{\dfrac{SQ_{E}}{a\cdot\left(n-1\right)}}=
\dfrac{\dfrac{n \cdot \displaystyle\sum_{i=1}^{a=2}\left(\bar{y}_{i\cdot} - \bar{y}_{\cdot\cdot}\right)^2}{a-1}}{\dfrac{\displaystyle\sum_{i=1}^{a=2} \sum_{j=1}^{n=25}\left(y_{ij}-\bar{y}_{i\cdot}\right)^2}{a\cdot\left(n-1\right)}}=
    \dfrac{MQ_{tratamentos}}{MQ_E}
}
$$  

Para iniciarmos os calculos vamos primeiramente calular as média e armazena-las

In [6]:
mean_roug_mat = dados_roughness_material.groupby('material').mean()   # media da rugosidade por material
mean_roug_mat

,roughness
material,
abs,193.44
pla,147.72


In [7]:
mean_roug_mat_gen = dados_roughness_material['roughness'].mean()  # Média Geral 
mean_roug_mat_gen

170.58

Calculando o $SQ_{tratamentos}$:

In [8]:
n = (dados_roughness_material['material'].value_counts()).loc['abs']
SQ_trat = n * sum((mean_roug_mat['roughness']-mean_roug_mat_gen)**2)
SQ_trat

26128.980000000003

Calculando $SQ_E$

In [9]:
SQ_abs = sum((dados_roughness_material[dados_roughness_material['material']=='abs']['roughness']-float(mean_roug_mat.loc['abs']))**2)
SQ_pla = sum((dados_roughness_material[dados_roughness_material['material']=='pla']['roughness']-float(mean_roug_mat.loc['pla']))**2)

SQ_e = SQ_abs+SQ_pla
SQ_e

454451.19999999995

Com os erros quadráticos calculados podemos fazer o teste de hipótese:  

In [10]:
a = len(mean_roug_mat)

MQ_trat = SQ_trat/(a-1)
MQe = SQ_e/(a*(n-1))

F0 = MQ_trat/MQe

F_bicaudal_hypoteses(F0, 0.05, a-1, a*(n-1))

O valor da estatísca cálcula F0 é: 2.7598
O limites da distribuição para ɑ de 0.05 é: [0.0010:5.3541]
Portanto, FALHA EM REJEITAR a hipótese nula!


Como podemos observar para um $\alpha$ de 0,05 FALHAMOS EM REJEITAR a hipótese nula, logo o material não afeta estatísticamente no valor de rugosidade.   
Além do teste ANOVA utilizaremos então o método MDS para avaliar se a as médias medidas tem diferença significativas na rugosidade.  
Para isso temos que para as médias terem diferenças significativas devemos relacionar:  
$$
\mathsf{
    |\bar{y}_{i\cdot}-\bar{y}_{j\cdot}|> MDS
}
$$  
$$
\mathsf{
    |\bar{y}_{i\cdot}-\bar{y}_{j\cdot}|> \mathit{t}_{\frac{\alpha}{2};a \cdot(n-1)} \cdot \sqrt{\dfrac{2\cdot MQ_E}{n}}
}
$$  
Assim:  

In [11]:
MDS = st.t.ppf(1-0.025,a*(n-1)) * sqrt((2*MQe)/n)

print('{:^10}|{:^10}'.format('abs-pla','MDS'))
print('{:^1}{:.4f}{:^2}|{:^1}{:.4f}{:^2}'.format('',float(mean_roug_mat.loc['abs'] - mean_roug_mat.loc['pla']),'','', MDS,''))

 abs-pla  |   MDS    
 45.7200  | 55.3352  


Analisando a mínima diferença significativa (MDS), vemos que realmente a mudança do material não afeta estatistiamente o valor da rugosidade.  
Essa conclusão era esperada, uma vez que como a impressão ocorre por camadas, a rugosidade deve ser afeta prinicpalmente por parâmetros que determinam as caracteristicas dessas camadas que o material em si.  

### ***Analise do preenchimento interno***  
Com a influência do material sendo descartada agora avaliaremos se o preenchimento interno da peça influencia na rugosidade da peça.  
Primeiramente vamos separar os dados de preenchimento interno e rugosidade.   

In [12]:
pattern_roughness = dados[['infill_pattern','roughness']]
pattern_roughness.head()

,infill_pattern,roughness
0,grid,25
1,honeycomb,32
2,grid,40
3,honeycomb,68
4,grid,92


In [13]:
pattern_roughness['infill_pattern'].value_counts()

honeycomb    25
grid         25
Name: infill_pattern, dtype: int64

Separado os dados vamos então realizar um teste ANOVA seguindo as hipóteses:  
$$
\left\{
    \begin{array}{}
        \mathsf{\mathit{H_0}: \tau_{0}=\tau_{1}=0}\\
        \mathsf{\mathit{H_1}: \tau_{0}\not=\tau_{1}\not=0}
    \end{array}
\right.
$$  
Para isso assumimos que o modelo assumi uma estatística F, assim:  
$$
\mathsf{
    F_0~=~\dfrac{\dfrac{SQ_{tratamentos}}{a-1}}{\dfrac{SQ_{E}}{a\cdot\left(n-1\right)}}=
\dfrac{\dfrac{n \cdot \displaystyle\sum_{i=1}^{a=2}\left(\bar{y}_{i\cdot} - \bar{y}_{\cdot\cdot}\right)^2}{a-1}}{\dfrac{\displaystyle\sum_{i=1}^{a=2} \sum_{j=1}^{n=25}\left(y_{ij}-\bar{y}_{i\cdot}\right)^2}{a\cdot\left(n-1\right)}}=
    \dfrac{MQ_{tratamentos}}{MQ_E}
}
$$  
Assim calculamos $F_0$ e fazemos o teste de hipótese:  

In [14]:
patt_rough_mean = pattern_roughness.groupby('infill_pattern').mean()
patt_rough_mean

,roughness
infill_pattern,
grid,177.28
honeycomb,163.88


In [15]:
patt_rough_gen = pattern_roughness.mean()
patt_rough_gen

roughness    170.58
dtype: float64

In [16]:
n_ptt = (pattern_roughness['infill_pattern'].value_counts()).loc['grid']
SQ_tra_ptt = n_ptt * sum( (patt_rough_mean['roughness'] - float(patt_rough_gen))**2 )
SQ_tra_ptt

2244.500000000002

In [23]:
SQ_e_grid = sum((pattern_roughness[pattern_roughness['infill_pattern']=='grid']['roughness'] - float(patt_rough_mean.loc['grid']))**2)
SQ_e_hone = sum((pattern_roughness[pattern_roughness['infill_pattern']=='honeycomb']['roughness'] - float(patt_rough_mean.loc['honeycomb']))**2)

SQ_e_ptt = SQ_e_grid + SQ_e_hone
SQ_e_ptt

478335.67999999993

In [26]:
a_ptt = len(patt_rough_mean)

MQ_trat_ptt = SQ_tra_ptt/(a-1)
MQe_ptt = SQ_e_ptt/(a*(n-1))

F0_ptt = MQ_trat_ptt/MQe_ptt

F_bicaudal_hypoteses(F0, 0.05, a-1, a*(n-1))

O valor da estatísca cálcula F0 é: 2.7598
O limites da distribuição para ɑ de 0.05 é: [0.0010:5.3541]
Portanto, FALHA EM REJEITAR a hipótese nula!


#### ***Modelo de Regressão Linear Multipla***  
Com os dados restantes vamos fazer um modelo de regressão multipla com 7 variáveis $\mathsf{\left(x_1, x_2, x_3, x_4, x_5, x_6, x_7\right)}$, sendo cada uma dessas uma entrada medida restante, e possui 8 coeficientes $\mathsf{\left(\hat{\beta}_0, \hat{\beta}_1, \hat{\beta}_2, \hat{\beta}_3, \hat{\beta}_4, \hat{\beta}_5, \hat{\beta}_6, \hat{\beta}_7\right)}$.  
Primeiramente vamos separar os dados com relação com a rugosidade:  

In [28]:
data_roughness = dados.drop(['infill_pattern', 'material', 'tension_strenght', 'elongation'], axis=1)
data_roughness.head()

,layer_height,wall_thickness,infill_density,nozzle_temperature,bed_temperature,print_speed,fan_speed,roughness
0,0.02,8,90,220,60,40,0,25
1,0.02,7,90,225,65,40,25,32
2,0.02,1,80,230,70,40,50,40
3,0.02,4,70,240,75,40,75,68
4,0.02,6,90,250,80,40,100,92


***
#### Teoria 📖  
Para determinarmos os coeficiente usaremos a seguinte relação: 
$$
\mathsf{
    \hat{\beta} = \left( X' \cdot X \right)^{-1} \cdot X' \cdot y
}
$$  
Onde:  
$$
\mathsf{y} = \left[
    \begin{array}{c}
        \mathsf{y_1}\\ \mathsf{y_2}\\ \vdots \\ \mathsf{y_n}
    \end{array}
\right];

\mathsf{X} = \left[
    \begin{array}{ccccc}
        1 & \mathsf{x_{11}} & \mathsf{x_{12}} & \cdots & \mathsf{x_{1k}}\\
        1 & \mathsf{x_{21}} & \mathsf{x_{22}} & \cdots & \mathsf{x_{2k}}\\
        \vdots & \vdots & \vdots & \vdots & \vdots\\
        1 & \mathsf{x_{11}} & \mathsf{x_{12}} & \cdots & \mathsf{x_{1k}}
    \end{array}
\right]; 

\mathsf{\hat{\beta}} = \left[
    \begin{array}{c}
        \mathsf{\beta_0}\\ \mathsf{\beta_1}\\ \vdots \\ \mathsf{\beta_0}
    \end{array}
\right]
$$  

Desta equação temos a matriz de covariância que é:  
$$
\mathsf{
    C = \left(X' \cdot X \right)^{-1}
}
$$  
***
Primeiramente vamos determinar a matriz de covariância:  

In [65]:
X_roug = np.array(data_roughness.drop('roughness',axis=1))
Y_roug = np.array(data_roughness['roughness'])

C_roug = np.linalg.inv((X_roug.T).dot(X_roug))
C_roug

array([[ 5.24386052e+00,  3.13062506e-02, -3.77559096e-04,
         1.21916643e-03, -1.79810101e-02,  1.89362315e-03,
         3.36654541e-03],
       [ 3.13062506e-02,  3.18852006e-03, -4.09108343e-05,
         1.25746058e-04, -8.81647394e-04,  1.32146819e-04,
         1.52545413e-04],
       [-3.77559096e-04, -4.09108343e-05,  3.57060981e-05,
        -2.41652827e-05,  5.54878870e-05,  1.14141170e-06,
        -5.15444272e-06],
       [ 1.21916643e-03,  1.25746058e-04, -2.41652827e-05,
         1.65267613e-04, -5.71139492e-04,  3.39295021e-06,
         7.32127855e-05],
       [-1.79810101e-02, -8.81647394e-04,  5.54878870e-05,
        -5.71139492e-04,  2.12969904e-03, -5.87946533e-05,
        -2.98604223e-04],
       [ 1.89362315e-03,  1.32146819e-04,  1.14141170e-06,
         3.39295021e-06, -5.87946533e-05,  2.89252055e-05,
         1.12278455e-05],
       [ 3.36654541e-03,  1.52545413e-04, -5.15444272e-06,
         7.32127855e-05, -2.98604223e-04,  1.12278455e-05,
         6.0450423

Com a matriz de covariância podemos calcular os coeficientes da regressão:  

In [66]:
B_rough = (C.dot(X_roug.T)).dot(Y_roug)

for i in range(len(B_rough)):
    print('{}{}: {:.4f}'.format('\u03B2', i, B_rough[i]))

β0: 1262.8094
β1: 1.6442
β2: 0.1694
β3: 2.3852
β4: -9.1175
β5: 0.6373
β6: 1.7639


Assim o modelo de multiplas variáveis dado é:  
$$
\mathsf{
    \hat{y}_{rugosidade} = 1262,8094 + 1,6442 \cdot x_1 + 0,1694 \cdot x_2 + 2,3852 \cdot x_3 - 9,1175 \cdot x_4 + 0,6373 \cdot x_5 + 1,7639 \cdot x_6
}
$$